In [1]:
import urllib.request
import urllib.parse
import json
import pprint
from pprint import PrettyPrinter

# === Step 1: API トークンの取得 ===
token_url = 'http://localhost:18080/kabusapi/token'
auth_data = { 'APIPassword': '1995taka' }  # 自身の API パスワードを設定
json_auth_data = json.dumps(auth_data).encode('utf8')

token_req = urllib.request.Request(token_url, data=json_auth_data, method='POST')
token_req.add_header('Content-Type', 'application/json')

try:
    with urllib.request.urlopen(token_req) as token_res:
        print("トークン取得レスポンス:", token_res.status, token_res.reason)
        token_content = json.loads(token_res.read())
        pprint.pprint(token_content)
except urllib.error.HTTPError as e:
    print("トークン取得 HTTP エラー:", e)
    try:
        error_content = json.loads(e.read())
        pprint.pprint(error_content)
    except Exception as parse_err:
        print("エラー解析中に問題発生:", parse_err)
    exit(1)
except Exception as e:
    print("トークン取得 エラー:", e)
    exit(1)

# API トークンの抽出
api_token = token_content.get('Token')
if not api_token:
    print("トークンが取得できませんでした。")
    exit(1)

# === Step 2: ポジション情報の取得 ===
positions_url = 'http://localhost:18080/kabusapi/positions'
# パラメータ設定（必要に応じて変更）
params = {
    'product': 2,      # 0:すべて
    'symbol': '9432'  # 例: '9433'
    # 'side': '1',       # 1:売、2:買
    # 'addinfo': 'false' # 追加情報を出力しない
}
query_string = urllib.parse.urlencode(params)
full_positions_url = f"{positions_url}?{query_string}"

positions_req = urllib.request.Request(full_positions_url, method='GET')
positions_req.add_header('Content-Type', 'application/json')
positions_req.add_header('X-API-KEY', api_token)

try:
    with urllib.request.urlopen(positions_req) as pos_res:
        print("\nポジション取得レスポンス:", pos_res.status, pos_res.reason)
        for header in pos_res.getheaders():
            print(header)
        print()
        positions_content = json.loads(pos_res.read())

        # 取得内容の確認のため、PrettyPrinterを使用
        pp = PrettyPrinter(sort_dicts=False)

        # ポジション情報がリスト形式の場合、最新の2件を抽出
        if isinstance(positions_content, list) and len(positions_content) >= 2:
            latest2_positions = positions_content[-3:]
        else:
            latest2_positions = positions_content

        print("\n最新2件のポジション:")
        pp.pprint(latest2_positions)

except urllib.error.HTTPError as e:
    print("ポジション取得 HTTP エラー:", e)
    try:
        error_content = json.loads(e.read())
        pprint.pprint(error_content)
    except Exception as parse_err:
        print("エラー応答解析エラー:", parse_err)
except Exception as e:
    print("その他のエラー:", e)


トークン取得レスポンス: 200 OK
{'ResultCode': 0, 'Token': 'aa6b7ce179f84206accebab429fc294a'}

ポジション取得レスポンス: 200 OK
('Content-Length', '58977')
('Content-Type', 'application/json; charset=utf-8')
('Server', 'Microsoft-HTTPAPI/2.0')
('X-Content-Type-Options', 'nosniff')
('X-XSS-Protection', '1; mode=block')
('X-Frame-Options', 'DENY')
('Content-Security-Policy', "default-src 'none'")
('Access-Control-Allow-Origin', '*')
('Date', 'Thu, 23 Jan 2025 06:11:37 GMT')
('Connection', 'close')


最新2件のポジション:
[{'MarginTradeType': 3,
  'ExecutionDay': 20250123,
  'Expenses': 0.0,
  'Commission': 0.0,
  'CommissionTax': 0.0,
  'ExpireDay': 20250123,
  'ExecutionID': 'E2025012303KWU',
  'AccountType': 4,
  'Symbol': '9432',
  'SymbolName': '日本電信電話',
  'Exchange': 1,
  'ExchangeName': '東証プ',
  'Price': 151.4,
  'LeavesQty': 0.0,
  'HoldQty': 0.0,
  'Side': '2',
  'CurrentPrice': 151.5,
  'Valuation': 0.0,
  'ProfitLoss': None,
  'ProfitLossRate': None},
 {'MarginTradeType': 3,
  'ExecutionDay': 20250123,
  'Expe